In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, LSTM, Embedding
data = pd.read_csv('IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [0]:
def preprocess(text):
    review = re.compile(r'<[^>]+>').sub('', text)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = re.sub(r"\s+[a-zA-Z]\s+", ' ', review)
    review = re.sub(r'\s+', ' ', review)
    return review

In [0]:
data['review'] = data.review.apply(lambda x:preprocess(x))
data['sentiment'] = data.sentiment.apply(lambda x:1 if x=='positive' else 0)
train_review, valid_review, train_sentiment, valid_sentiment = train_test_split(data.review.values, data.sentiment.values, test_size=0.20, random_state=42)
tokenizer = Tokenizer(num_words=5000, lower=True, oov_token='<OOV>')
tokenizer.fit_on_texts(train_review)
train_review = tokenizer.texts_to_sequences(train_review)
valid_review = tokenizer.texts_to_sequences(valid_review)

In [0]:
train_review = pad_sequences(train_review, padding='post', maxlen=200)
valid_review = pad_sequences(valid_review, padding='post', maxlen=200)

In [5]:
model = Sequential() 
model.add(Embedding(5000, 64, input_length=200)) 
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 64)           320000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 418,945
Trainable params: 418,945
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(train_review, train_sentiment, validation_data=(valid_review, valid_sentiment), epochs=20, batch_size=128) 

Epoch 1/20
313/313 [==============================] - 18s 56ms/step - loss: 0.5696 - accuracy: 0.7048 - val_loss: 0.6364 - val_accuracy: 0.6486
Epoch 2/20
313/313 [==============================] - 17s 54ms/step - loss: 0.4399 - accuracy: 0.8257 - val_loss: 0.4285 - val_accuracy: 0.8457
Epoch 3/20
313/313 [==============================] - 17s 53ms/step - loss: 0.3776 - accuracy: 0.8609 - val_loss: 0.3683 - val_accuracy: 0.8597
Epoch 4/20
313/313 [==============================] - 17s 53ms/step - loss: 0.3307 - accuracy: 0.8743 - val_loss: 0.3033 - val_accuracy: 0.8797
Epoch 5/20
313/313 [==============================] - 17s 54ms/step - loss: 0.3178 - accuracy: 0.8801 - val_loss: 0.3145 - val_accuracy: 0.8724
Epoch 6/20
313/313 [==============================] - 17s 54ms/step - loss: 0.3166 - accuracy: 0.8785 - val_loss: 0.3203 - val_accuracy: 0.8802
Epoch 7/20
313/313 [==============================] - 17s 53ms/step - loss: 0.2955 - accuracy: 0.8886 - val_loss: 0.3602 - val_accuracy:

In [27]:
from sklearn.metrics import accuracy_score
evaluate = model.evaluate(valid_review, valid_sentiment, verbose=1)
print("Accuracy on validation dataset:", evaluate[1])

313/313 [==============================] - 5s 16ms/step - loss: 0.3177 - accuracy: 0.8783
Test Accuracy: 0.8783000111579895
